# YOLOv5 implementation

Dataset link: [chest-anatomy-yolov5-train-set](https://www.kaggle.com/davidbroberts/chest-anatomy-yolov5-train-set)

Ref: [Do_Thuan.pdf](https://www.theseus.fi/bitstream/handle/10024/452552/Do_Thuan.pdf)

## 1. Check GPU usage status:
To use GPU: Go to Edit -> Notebook settings -> Hardware accelerator: GPU.
If we don't use GPU, we can skip this step.

In [ ]:
!nvidia-smi

## 2. Clone yolov5 repository:

In [ ]:
# !pip3 install torch==1.9.1+cu102 torchvision==0.10.1+cu102 torchaudio===0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

> Install torch with stable version to be able to run code with CPU. But the training is VERY slow 😭. 

> After we use GPU a period of time, we will exceed usage limit, then we will have to wait a long time for usage limit to be reseted.

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 10382, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 10382 (delta 0), reused 3 (delta 0), pack-reused 10379
Receiving objects: 100% (10382/10382), 10.59 MiB | 14.05 MiB/s, done.
Resolving deltas: 100% (7165/7165), done.


In [ ]:
# If we use GPU then use default requirement file
!pip3 install -r /content/yolov5/requirements.txt
# !pip3 install -r requirements.txt
%cd /content/yolov5

## 3. Import neccessary libraries:

In [ ]:
import torch
import os
import shutil
import numpy as np
from IPython.display import Image, clear_output

clear_output()
print('Setup complete')

Setup complete


> If you have clone the repo twice, you can't delete folder directly. Insteads, use this code to delete the folder.

> ⚠️ Note: Use this code wisely.

In [ ]:
import shutil

# shutil.rmtree('/content/yolov5/runs/')

## 4. Mount your drive for uploaded data:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## 5. Edit data.yaml file:

In [ ]:
root_dir = '/content/drive/MyDrive/anatomy/'
num_classes = 10
data_dir = root_dir + 'chest_anatomy_data.yaml'
with open(data_dir, "w") as dataFile:
  dataFile.write(f"""
path: {root_dir}
train: images/train
val: images/test

# Classes
nc: {num_classes}
names: ['lung_left','lung_right','clavicle_left','clavicle_right','humeral_head_left','humeral_head_right','heart','gastric_bubble','carina','trachea']
""")

In [ ]:
%cat /content/drive/MyDrive/anatomy/chest_anatomy_data.yaml

## 6. Create a config file:

In [ ]:
root_dir = '/content/drive/MyDrive/anatomy/'
config_dir = root_dir + 'custom_config.yaml'
with open(config_dir, "w") as dataFile:
  dataFile.write(f"""
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]
""")

In [ ]:
%cat /content/drive/MyDrive/anatomy/custom_config.yaml

## 7. Train model:

> Since we don't train model with pre-trained model, the training process is very slow.

In [ ]:
%%time
%cd /content/yolov5/

!python train.py --img 416 --batch 32 --epochs 100 --data /content/drive/MyDrive/anatomy/chest_anatomy_data.yaml --cfg /content/drive/MyDrive/anatomy/custom_config.yaml --weights '' --name anatomy

/content/yolov5
train: weights=, cfg=/content/drive/MyDrive/anatomy/custom_config.yaml, data=/content/drive/MyDrive/anatomy/chest_anatomy_data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=100, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=anatomy, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-163-gd95978a torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, a

## 8. Inference with trained weight:


> ⚠️ Warning: Don't use a youtube link to detect. Because, Youtube recently removed the dislike features, which makes the library used by YOLOv5 broken.

For each model and dataset, adjust an appropriate confidence score.

In [ ]:
%cd /content/yolov5/
!python detect.py --weight '/content/yolov5/runs/train/anatomy7/weights/best.pt' --data /content/drive/MyDrive/anatomy/chest_anatomy_data.yaml --img 416 --conf 0.03 --source '/content/drive/MyDrive/anatomy/images/test'

/content/yolov5
detect: weights=['/content/yolov5/runs/train/anatomy7/weights/best.pt'], source=/content/drive/MyDrive/anatomy/images/test, data=/content/drive/MyDrive/anatomy/chest_anatomy_data.yaml, imgsz=[416, 416], conf_thres=0.03, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-163-gd95978a torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 367 layers, 46156743 parameters, 0 gradients, 107.9 GFLOPs
image 1/20 /content/drive/MyDrive/anatomy/images/test/5b1e5fabcb5d.jpg: 352x416 Done. (0.062s)
image 2/20 /content/drive/MyDrive/anatomy/images/test/5b322d104117.jpg: 416x352 3 lung_lefts, 6 clavicle_lefts, 9 clavicle_rights, 1 humeral_head_right, 2 hearts, Done. (0.080s)
i